In [1]:
import json
import numpy as np
import os
import tensorflow as tf

In [2]:
# Turns all the json files in a folder into numpy arrays
def json_to_nparray():
    for filename in os.listdir('Dataset_tokenized_BPE'):
        if filename.endswith('.json'):
            with open('Dataset_tokenized_BPE/' + filename) as f:
                data = json.load(f)
                np.save('Dataset_tokenized_BPE/' + filename[:-5], np.array(data))
            continue
        else:
            continue
        
# revert the above function to get the json files back from the numpy arrays
def nparray_to_json():
    for filename in os.listdir('Dataset_tokenized_BPE'):
        if filename.endswith('.npy'):
            data = np.load('Dataset_tokenized_BPE/' + filename)
            with open('Dataset_tokenized_BPE/' + filename[:-4] + '.json', 'w') as f:
                json.dump(data.tolist(), f)
            continue
        else:
            continue
        
# open .npy files from a folder and concatenate them into one numpy array
def npy_to_nparray():   
    data = []
    for filename in os.listdir('Dataset_tokenized_BPE'):
        if filename.endswith('.npy'):
            data.append(np.load('Dataset_tokenized_BPE/' + filename))
            continue
        else:
            continue
    return data

In [3]:
# open .npy files from a folder and concatenate all the tokens into one numpy array
tokens = np.array(()) # empty numpy array
for filename in os.listdir('Dataset_tokenized_BPE'): # iterate through all files in the folder
    if filename.endswith('.npy'): # if the file is a .npy file
        for i in range(len(np.load(('Dataset_tokenized_BPE/' + filename), allow_pickle=True)[()].get('tokens'))): # iterate through all the tokens in the file
            tokens = np.append(tokens, np.load(('Dataset_tokenized_BPE/' + filename), allow_pickle=True)[()].get('tokens')[i]) # append the tokens to the numpy array
        continue 
    else: 
        continue 
    
# revert the above funtion to get the .npy files back from the numpy array
def nparray_to_npy():
    # create a folder to store the .npy files
    if not os.path.exists('Dataset_tokenized_BPE'):
        

In [4]:
tokens.shape

(1819900,)

In [5]:
"""
x = train_data[:BLOCK_SIZE]
y = train_data[1:BLOCK_SIZE+1]
for t in range(BLOCK_SIZE):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

xb , yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('----')


for b in range(BATCH_SIZE): # batch dimension
    for t in range(BLOCK_SIZE): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.numpy().tolist()} the target: {target}")#
"""

data = tf.cast(tokens, tf.int32) # put tokens into a tensor and cast to int32
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n] # first 90% of data
val_data = data[n:] # last 10% of data

BLOCK_SIZE = 12
BATCH_SIZE = 2
    
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = tf.random.uniform((BATCH_SIZE,), maxval=len(data)-BLOCK_SIZE, dtype=tf.int32)
    x = tf.stack([data[i:i+BLOCK_SIZE] for i in ix])
    y = tf.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])
    return x, y

In [6]:
vocab_size = data[np.argmax(tokens)] + 1
print(vocab_size)


tf.Tensor(500, shape=(), dtype=int32)


In [7]:
def prepare_batch(pt, en): 
    pt = tokenizers.pt.tokenize(pt)      # Output is ragged.
    pt = pt[:, :MAX_TOKENS]    # Trim to MAX_TOKENS.
    pt = pt.to_tensor()  # Convert to 0-padded dense Tensor

    en = tokenizers.en.tokenize(en)
    en = en[:, :(MAX_TOKENS+1)]
    en_inputs = en[:, :-1].to_tensor()  # Drop the [END] tokens
    en_labels = en[:, 1:].to_tensor()   # Drop the [START] tokens

    return (pt, en_inputs), en_labels

 
def make_batches(ds):
  return (
      ds
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(prepare_batch, tf.data.AUTOTUNE)
      .prefetch(buffer_size=tf.data.AUTOTUNE))


# positional encoding function 
def positional_encoding(length, depth):
  depth = depth/2

  positions = tf.range(length,dtype=tf.float32)[:, None]     # (seq, 1)
  depths = tf.range(depth,dtype=tf.float32)[None, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = tf.concat(
      [tf.math.sin(angle_rads), tf.math.cos(angle_rads)],
      axis=-1) 

  return pos_encoding

# positional embedding layer 
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True) 
    self.pos_encoding = positional_encoding(length=2048, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x
  
l = PositionalEmbedding(1000,512)

In [8]:
l(tf.keras.Input(shape=(16,),dtype = tf.int64))

<KerasTensor: shape=(None, 16, 512) dtype=float32 (created by layer 'positional_embedding')>

In [9]:
class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

class CrossAttention(BaseAttention):
  def call(self, x, context):
    attn_output, attn_scores = self.mha(
        query=x,
        key=context,
        value=context,
        return_attention_scores=True)

    # Cache the attention scores for plotting later.
    self.last_attn_scores = attn_scores

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

class GlobalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

class CausalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x,
        use_causal_mask = True)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x) 
    return x

class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()

    self.self_attention = GlobalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x):
    x = self.self_attention(x)
    x = self.ffn(x)
    return x

class Encoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads,
               dff, vocab_size, dropout_rate=0.1):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(
        vocab_size=vocab_size, d_model=d_model)

    self.enc_layers = [
        EncoderLayer(d_model=d_model,
                     num_heads=num_heads,
                     dff=dff,
                     dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x):
    # `x` is token-IDs shape: (batch, seq_len)
    x = self.pos_embedding(x)  # Shape `(batch_size, seq_len, d_model)`.

    # Add dropout.
    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x)

    return x  # Shape `(batch_size, seq_len, d_model)`.

class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self,
               *,
               d_model,
               num_heads,
               dff,
               dropout_rate=0.1):
    super(DecoderLayer, self).__init__()

    self.causal_self_attention = CausalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.cross_attention = CrossAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x):
    x = self.causal_self_attention(x)
    x = self.cross_attention(x,x)

    # Cache the last attention scores for plotting later
    self.last_attn_scores = self.cross_attention.last_attn_scores

    x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
    return x

class Decoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size,
               dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size,
                                             d_model=d_model)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads,
                     dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]

    self.last_attn_scores = None

  def call(self, x):
    # `x` is token-IDs shape (batch, target_seq_len)
    x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

    x = self.dropout(x)

    for i in range(self.num_layers):
      x  = self.dec_layers[i](x)

    self.last_attn_scores = self.dec_layers[-1].last_attn_scores

    # The shape of x is (batch_size, target_seq_len, d_model).
    return x

class Transformer(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff,
               input_vocab_size, target_vocab_size, dropout_rate=0.1):
    super().__init__()

    self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=target_vocab_size,
                           dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inputs):
    # To use a Keras model with `.fit` you must pass all your inputs in the
    # first argument.
    x  = inputs

    x = self.decoder(x)  # (batch_size, target_len, d_model)

    # Final linear layer output.
    logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

    try:
      # Drop the keras mask, so it doesn't scale the losses/metrics.
      # b/250038731
      del logits._keras_mask
    except AttributeError:
      pass

    # Return the final output and the attention weights.
    return logits


In [10]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1
MAX_TOKENS = 10
BUFFER_SIZE = 20000
BATCH_SIZE = 2

In [11]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=vocab_size,
    target_vocab_size=vocab_size,
    dropout_rate=dropout_rate)

In [12]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

def masked_loss(label, pred):
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss


def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)


In [13]:
a ,b = get_batch('train')

In [14]:
def train_ds_gen():
    for i in range(1000):
        yield get_batch('train')
        
def val_ds_gen():
    for i in range(100):
        yield get_batch('train')
        
train_ds= tf.data.Dataset.from_generator(train_ds_gen, output_types=(tf.int32, tf.int32))
val_ds = tf.data.Dataset.from_generator(val_ds_gen, output_types=(tf.int32, tf.int32))

In [15]:
a = tf.keras.Input(shape=(16,256))
l = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=64)(a, a,a,use_causal_mask = True)
l

<KerasTensor: shape=(None, 16, 256) dtype=float32 (created by layer 'multi_head_attention_8')>

In [16]:
transformer(get_batch('train')[0])

<tf.Tensor: shape=(2, 12, 500), dtype=float32, numpy=
array([[[ 0.41843367, -0.6640315 , -0.9288635 , ..., -0.18519644,
         -0.21592776, -0.15706   ],
        [ 0.20486599, -1.1446135 , -0.23492339, ..., -0.14431347,
         -0.05127469, -0.03966749],
        [ 0.28998542, -0.7843472 , -0.4523071 , ...,  0.45857686,
         -1.1024239 ,  0.01032036],
        ...,
        [-0.04622516, -0.1424579 ,  0.12012563, ...,  1.1061985 ,
         -1.536612  ,  0.40399054],
        [ 0.49052036, -0.7470296 ,  0.11027565, ...,  0.2995183 ,
         -0.22417507,  0.4130093 ],
        [-0.09576005,  0.3553278 ,  0.04561641, ...,  0.7936692 ,
         -1.3002354 ,  0.47641712]],

       [[ 0.47951475, -0.93732077, -0.5168369 , ..., -0.3730178 ,
         -0.20323567, -0.05597189],
        [ 0.9160687 , -1.0253688 , -0.22526588, ..., -0.2322132 ,
          0.1306046 , -0.13766426],
        [ 0.56257695, -0.96330166, -0.6565111 , ...,  0.59725106,
         -0.0322721 ,  0.01379591],
        ...,


In [17]:
transformer.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])

transformer.fit(train_ds,
                epochs=20,
                validation_data=val_ds)


Epoch 1/20


ValueError: in user code:

    File "c:\Users\benja\Anaconda3\envs\iannwtf\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\benja\Anaconda3\envs\iannwtf\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\benja\Anaconda3\envs\iannwtf\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\benja\Anaconda3\envs\iannwtf\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\benja\Anaconda3\envs\iannwtf\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\benja\AppData\Local\Temp\__autograph_generated_file4jgx6f_3.py", line 11, in tf__call
    